<a href="https://colab.research.google.com/github/SChoi005/DeepLearningBasics/blob/main/tensorflow_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Lite
1. Training a model on a Computer
2. Optimizing a model with TF Life
3. Inference time on a moble device

* tensorflow 2.x doesn't apply tensorflow lite

## Installing dependencies and setting up the environment

In [1]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220527125636.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636


In [2]:
!pip install tensorflow-gpu==1.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 345.0 MB 4.3 kB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
     |████████████████████████████████| 367 kB 73.5 MB/s 
     |████████████████████████████████| 3.2 MB 53.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0


## Importing project dependencies

In [1]:
import tensorflow as tf
import numpy as np
 
from tensorflow.keras.datasets import fashion_mnist

tf.__version__

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.13.1'

## Dataset preprocessing

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
# Image normalization
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# Reshaping of the dataset
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

## Building a model

In [5]:
model = tf.keras.models.Sequential()

In [6]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [8]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.5260 - sparse_categorical_accuracy: 0.8116
Epoch 2/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3987 - sparse_categorical_accuracy: 0.8554
Epoch 3/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.3667 - sparse_categorical_accuracy: 0.8656
Epoch 4/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.3451 - sparse_categorical_accuracy: 0.8733
Epoch 5/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.3301 - sparse_categorical_accuracy: 0.8790


In [9]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy : {}".format(test_accuracy))

10000/10000 [==============================] - 0s 45us/sample - loss: 0.3570 - sparse_categorical_accuracy: 0.8699
Test accuracy : 0.8698999881744385


## Converting the model into TensorFlow Lite version

In [10]:
# Saving the model
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

In [11]:
# Creating the TFLite Converter
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


In [12]:
# Converting the model
tflite_model = converter.convert()

In [13]:
# Saving the TFLite version of the model
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)